In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.core.display import display, HTML
import os

from src import *
pd.set_option('display.max_rows', None)

# Parameters

These parameters configure various things such as the relative location of data files, which date is being analysed
and the assumed generation period for an infection.

In [ ]:
ALL_VIC_CASES='archive/2021-07-12/all-vic-cases.csv'
QUARANTINE='archive/2021-07-16/quarantine.csv'
GENERATION_DAYS=5
ENABLE_MELBOURNE_ANIMATION=False

TODAY=datetime.today().strftime("%Y-%m-%d")
YESTERDAY=add_days(TODAY, -1)
LAST_WEEK=add_days(TODAY, -7)
RUN_DATE=TODAY
LOAD_DATE=RUN_DATE

PREV_DAY=add_days(RUN_DATE, -1)
NEXT_WEEK=add_days(RUN_DATE, 7)
PREV_WEEK=add_days(RUN_DATE, -7)
PREV_FORTNIGHT=add_days(RUN_DATE, -14)

NSW_6_MONTHS=lambda date: f"archive/{date}/last-6-months-nsw.csv"
NSW_14_DAYS=lambda date: f"archive/{date}/last-14-days-nsw.csv"

AMEND_TOTAL=None

# Data Preparation

**sweep_downloads()** moves files from the $HOME/Downloads directory into today's archive directory. Thses should first be
downloaded from the "Last 14 days (new)" and "Last 6 months (true)" panels of the NSW Transmission Sources section of [covid19data.com.au](https://www.covid19data.com.au/nsw)

In [ ]:
sweep_downloads(TODAY)
if not os.path.exists(NSW_6_MONTHS(LOAD_DATE)):
    LOAD_DATE=YESTERDAY
assert os.path.exists(NSW_6_MONTHS(LOAD_DATE))

In [ ]:
nsw_df = update_df(load_data(NSW_6_MONTHS(LOAD_DATE)),load_data(NSW_14_DAYS(LOAD_DATE)))
if AMEND_TOTAL:
    nsw_df = append_recent_date(nsw_df, RUN_DATE, AMEND_TOTAL)
vic_df = load_vic_data(ALL_VIC_CASES)
quarantine_df = load_quarantine(QUARANTINE)

Next, we truncate and refindex the data frames to each outbreak.

In [ ]:
avalon = select_outbreak(nsw_df[(nsw_df['date'] >= '2020-12-17') & (nsw_df['date'] <= '2021-01-16')], generation_days=GENERATION_DAYS)
bondi = select_outbreak(nsw_df[(nsw_df['date'] >= '2021-06-17')], generation_days=GENERATION_DAYS)
vic_outbreak = select_outbreak(vic_df[(vic_df['date'] >= '2020-05-27') & (vic_df['date'] <= '2020-10-29')], generation_days=GENERATION_DAYS)